* Model based recsys
    * ALS / Matrix factorization 으로 추천 결과 및 성능 비교하기

In [2]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
# 데이터셋 로드
data_path = './anime/'

anime = pd.read_csv(data_path+'anime.csv')
rating = pd.read_csv(data_path+'rating.csv')

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
rating.shape

(7813737, 3)

In [5]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


#### preprocessing

In [6]:
print(rating.shape)
rating['rating'].describe() # rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).

(7813737, 3)


count   7813737.00
mean          6.14
std           3.73
min          -1.00
25%           6.00
50%           7.00
75%           9.00
max          10.00
Name: rating, dtype: float64

In [7]:
rating.loc[rating['user_id']==1,'rating'].describe() # 어떤 user는 시청한 대부분의 item의 rating 을 매기지 않음

count   153.00
mean     -0.71
std       1.76
min      -1.00
25%      -1.00
50%      -1.00
75%      -1.00
max      10.00
Name: rating, dtype: float64

In [8]:
pd.merge(rating.loc[(rating['user_id']==1)&(rating['rating']>0)], anime, on ='anime_id', how='left')

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,8074,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
1,1,11617,10,High School DxD,"Comedy, Demons, Ecchi, Harem, Romance, School",TV,12,7.70,398660
2,1,11757,10,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100
3,1,15451,10,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",TV,12,7.87,266657


In [9]:
# 모든 item 의 rating 을 assign 하지 않은 경우? >> 3915명
# 이 경우에는 item feature 기반 contents based filtering 으로 추천
tmp =rating.groupby('user_id')['rating'].agg(['count','sum'])
tmp['tmp_val'] = tmp['count']+tmp['sum']  
tmp[tmp['tmp_val']==0]

,count,sum,tmp_val
user_id,,,
4,52,-52,0
6,37,-37,0
13,174,-174,0
49,6,-6,0
54,834,-834,0
...,...,...,...
73418,28,-28,0
73447,1,-1,0
73451,74,-74,0


In [10]:
exclude_users = list(tmp[tmp['tmp_val']==0].index)
train_users_interaction = rating.loc[~rating['user_id'].isin(exclude_users)]
print(train_users_interaction.shape)

(7422056, 3)


In [11]:
# 위 interaction dataframe 내 -1 rating 은 item dataframe 의 rating 으로 대체

In [12]:
train_users_interaction.loc[train_users_interaction['rating']==-1,'rating'] = None

In [13]:
len([c.strip() for c in anime['genre'].str.get_dummies(sep=',').columns])

82

In [14]:
anime_features = anime[['anime_id','episodes','rating','members']].join(anime['genre'].str.get_dummies(sep=', '))
anime_features.columns

Index(['anime_id', 'episodes', 'rating', 'members', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai',
       'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural',
       'Thriller', 'Vampire', 'Yaoi', 'Yuri'],
      dtype='object')

In [15]:
anime_features = pd.concat([anime_features, anime['type'].str.get_dummies()],axis=1)
anime_features.shape

(12294, 53)

In [16]:
genre_type_cols = list(set([c.strip() for c in anime_features.columns if c not in ['anime_id','episodes','rating','members']]))
print(len(genre_type_cols))
print(genre_type_cols)

48
['Shoujo', 'Romance', 'Comedy', 'Parody', 'Shounen Ai', 'Supernatural', 'Military', 'Slice of Life', 'TV', 'Harem', 'Cars', 'Samurai', 'Sports', 'Mystery', 'Special', 'Kids', 'Vampire', 'Josei', 'Yuri', 'Horror', 'Music', 'Dementia', 'Martial Arts', 'Demons', 'Sci-Fi', 'Hentai', 'Mecha', 'Shoujo Ai', 'Adventure', 'Action', 'Psychological', 'Ecchi', 'School', 'Yaoi', 'OVA', 'Thriller', 'Fantasy', 'Game', 'Drama', 'Space', 'Seinen', 'ONA', 'Magic', 'Super Power', 'Police', 'Shounen', 'Movie', 'Historical']


In [17]:
anime_features = anime[['anime_id','episodes','rating','members']].join(anime['genre'].str.get_dummies(sep=', '))
anime_features = pd.concat([anime_features, anime['type'].str.get_dummies()],axis=1)
genre_type_cols = list(set([c.strip() for c in anime_features.columns if c not in ['anime_id','episodes','rating','members']]))
print(len(genre_type_cols))
print(genre_type_cols)

anime_features.head()

48
['Shoujo', 'Romance', 'Comedy', 'Parody', 'Shounen Ai', 'Supernatural', 'Military', 'Slice of Life', 'TV', 'Harem', 'Cars', 'Samurai', 'Sports', 'Mystery', 'Special', 'Kids', 'Vampire', 'Josei', 'Yuri', 'Horror', 'Music', 'Dementia', 'Martial Arts', 'Demons', 'Sci-Fi', 'Hentai', 'Mecha', 'Shoujo Ai', 'Adventure', 'Action', 'Psychological', 'Ecchi', 'School', 'Yaoi', 'OVA', 'Thriller', 'Fantasy', 'Game', 'Drama', 'Space', 'Seinen', 'ONA', 'Magic', 'Super Power', 'Police', 'Shounen', 'Movie', 'Historical']


,anime_id,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,32281,1,9.37,200630,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,5114,64,9.26,793665,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,28977,51,9.25,114262,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9253,24,9.17,673572,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,9969,51,9.16,151266,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
total_df = pd.merge(train_users_interaction, anime_features, on='anime_id', how='left')
total_df.head()

,user_id,anime_id,rating_x,episodes,rating_y,members,Action,Adventure,Cars,Comedy,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,1,20,NaN,220,7.81,683297.00,1.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
1,1,24,NaN,26,8.06,178553.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
2,1,79,NaN,24,7.31,158772.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,1,226,NaN,13,7.85,623511.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
4,1,241,NaN,11,6.69,84395.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00


In [19]:
total_df['rating'] = np.where(total_df['rating_x'].isnull(), total_df['rating_y'],total_df['rating_x'])
del total_df['rating_x']
del total_df['rating_y']
total_df.head()

,user_id,anime_id,episodes,members,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV,rating
0,1,20,220,683297.00,1.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.81
1,1,24,26,178553.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8.06
2,1,79,24,158772.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.31
3,1,226,13,623511.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.85
4,1,241,11,84395.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6.69


In [20]:
total_df.groupby('user_id')['rating'].mean().rename('user_avg_rating').reset_index()

,user_id,user_avg_rating
0,1,7.50
1,2,8.51
2,3,7.59
3,5,4.41
4,7,7.39
...,...,...
69595,73512,8.52
69596,73513,7.52
69597,73514,10.00
69598,73515,8.45


In [21]:
total_df.head()

,user_id,anime_id,episodes,members,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV,rating
0,1,20,220,683297.00,1.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.81
1,1,24,26,178553.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8.06
2,1,79,24,158772.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.31
3,1,226,13,623511.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.85
4,1,241,11,84395.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6.69


In [22]:
def get_feature_set(df):
    
    X = df.drop(columns=['rating'])
    y = df[['rating']]
    print('# of original features :::: {}'.format(X.shape[1]))
    # feature engineering

    X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.25, random_state=42)

    # train set user based feature engineering

    # user baesd feature engineering
    user_feature_genre_rating = pd.concat([X_train,y_train],axis=1).reset_index(drop=True).melt(id_vars=['user_id', 'rating'], value_vars=genre_type_cols) \
    .query("value == 1") \
    .groupby(['user_id', 'variable'])['rating'].mean().unstack().fillna(0)

    user_feature_avg_rating = pd.concat([X_train,y_train],axis=1).groupby('user_id')['rating'].mean().rename('user_avg_rating').reset_index()
    user_feature_cnt_rating = pd.concat([X_train,y_train],axis=1).groupby('user_id')['rating'].count().rename('user_cnt_rating').reset_index()
    user_feature_std_rating = pd.concat([X_train,y_train],axis=1).groupby('user_id')['rating'].std().rename('user_std_rating').reset_index()

    # item based feature engineering
    item_feature_avg_rating = pd.concat([X_train,y_train],axis=1).groupby('anime_id')['rating'].mean().rename('item_avg_rating').reset_index()
    item_feature_cnt_rating = pd.concat([X_train,y_train],axis=1).groupby('anime_id')['rating'].count().rename('item_cnt_rating').reset_index()
    item_feature_std_rating = pd.concat([X_train,y_train],axis=1).groupby('anime_id')['rating'].std().rename('item_std_rating').reset_index()

    X_train_interaction = X_train[['user_id','anime_id']]
    X_train_concat = pd.merge(X_train_interaction,user_feature_genre_rating, on='user_id',how='left')
    X_train_concat = pd.merge(X_train_concat,user_feature_avg_rating, on='user_id',how='left')
    X_train_concat = pd.merge(X_train_concat,user_feature_cnt_rating, on='user_id',how='left')  
    X_train_concat = pd.merge(X_train_concat,user_feature_std_rating, on='user_id',how='left')
    X_train_concat = pd.merge(X_train_concat,item_feature_avg_rating, on='anime_id',how='left')
    X_train_concat = pd.merge(X_train_concat,item_feature_cnt_rating, on='anime_id',how='left')
    X_train_concat = pd.merge(X_train_concat,item_feature_std_rating, on='anime_id',how='left')

    
    X_test_interaction = X_test[['user_id','anime_id']]
    X_test_concat = pd.merge(X_test_interaction,user_feature_genre_rating, on='user_id',how='left')
    X_test_concat = pd.merge(X_test_concat,user_feature_avg_rating, on='user_id',how='left')
    X_test_concat = pd.merge(X_test_concat,user_feature_cnt_rating, on='user_id',how='left')  
    X_test_concat = pd.merge(X_test_concat,user_feature_std_rating, on='user_id',how='left')
    X_test_concat = pd.merge(X_test_concat,item_feature_avg_rating, on='anime_id',how='left')
    X_test_concat = pd.merge(X_test_concat,item_feature_cnt_rating, on='anime_id',how='left')
    X_test_concat = pd.merge(X_test_concat,item_feature_std_rating, on='anime_id',how='left')

    X_train_concat = X_train_concat.fillna(0.0)
    X_test_concat = X_train_concat.fillna(0.0)

    return X_train_concat, X_test_concat, y_train, y_test

In [23]:
X_train,X_test,y_train, y_test = get_feature_set(total_df)

# of original features :::: 53


In [24]:
X_train.to_pickle('./anime/X_train_anime.pkl')
X_test.to_pickle('./anime/X_test_anime.pkl')
y_train.to_pickle('./anime/y_train_anime.pkl')
y_test.to_pickle('./anime/y_test_anime.pkl')

In [3]:
X_train = pd.read_pickle('./anime/X_train_anime.pkl')
X_test = pd.read_pickle('./anime/X_test_anime.pkl')
y_train = pd.read_pickle('./anime/y_train_anime.pkl')
y_test = pd.read_pickle('./anime/y_test_anime.pkl')

In [4]:
user_item_encoder = OneHotEncoder(handle_unknown='ignore')  

X_train_interaction = user_item_encoder.fit_transform(X_train[['user_id','anime_id']].astype('category'))
X_test_interaction = user_item_encoder.transform(X_test[['user_id','anime_id']].astype('category'))

In [5]:
X_train_interaction

<5566542x78930 sparse matrix of type '<class 'numpy.float64'>'
	with 11133084 stored elements in Compressed Sparse Row format>

In [6]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train.drop(columns=['user_id','anime_id']))
X_test_sc = scaler.transform(X_test.drop(columns=['user_id','anime_id']))